In [1]:
import requests
import json
import datetime
import pandas as pd

In [2]:
def r_to_j(r):
    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
#    print(myJ_s)
    myJ = json.loads(myJ_s)
#    print(json.dumps(myJ_s, indent=1))
    return myJ


In [3]:
def do_r(req, page_size=2000, page_num=0,debug=False, attrs=[]):
    myJ = {}
    rJ = {'count': 0, 'g' : {}}
    keepGoing=1
    gDebug=0

    req += '&page_size='+str(page_size)+'&page_num='

    while keepGoing:
        keepGoing=1
        page_num += 1
        if page_num > 499:
            keepGoing = 0
        
        r = requests.get(req + str(page_num) )
        
#        if debug:
#            print(r, page_num, datetime.datetime.now())
        
        myJ = r_to_j(r)
        
        if gDebug:
            print('myj=', json.dumps(myJ, indent=1))
        
        if len(myJ['feed']['entry']) == 0:
            keepGoing=0
            continue

        rJ['count']+= len(myJ['feed']['entry'])
        outJ={}
        for j in myJ['feed']['entry']:
            if gDebug:
                if 'producer_granule_id' in j:
                    print('****************--->', j)
                else:
                    print ('producer_granule_id NOT FOUND!', j)
                gDebug=0
            outJ={}
            for a in attrs:
                if a in j:
                    outJ[a]=j[a]
                
            rJ['g'][j['id']]=outJ
            if debug:
                print ('*************************')
                print (json.dumps(j, indent=1))
                print ('*************************')
                break
        if debug:
            keepGoing=0

    return rJ

In [4]:
# Get a list of short_names from a provider
import time

my_provider='MODAPSNRT'
#my_provider='GES_DISC'
my_shortNames=[]
#print (r.headers)
myJ_s = ''
collectionsJ={}
uniqueKeysJ={}

page_num=0
keepGoing=1
debug=0
while keepGoing:
    
    page_num += 1
    if page_num > 200:
        keepGoing = 0
    print ('before request.get')  
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?platform=AQUA&sort_key=short_name&page_size=2&page_num='+str(page_num))
#    r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?provider=GES_DISC&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?provider=MODAPSNRT&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?instrument=MODIS&INSTRUMENT=MODIS&provider=LPDAAC_ECS&sort_key=short_name&page_size=2&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?granule_data_format=HDF-EOS2&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    print ('after request.get')  

    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
    #print(myJ_s)
    myJ = json.loads(myJ_s)
    #print(json.dumps(myJ, indent=1))

    if len(myJ['items']) == 0:
        keepGoing=0
        continue
    
    for r in myJ['items']:
        j=r['meta']
        #print ('j=', j)
        if debug == 1:
            print ('r=', json.dumps(r,indent=1))
            debug=0
        #for k in j:
        #    print ('meta',k, j[k])
        j=r['umm']
#        for k in j:
            #print ('umm', k)
#            if k == 'ShortName':
        my_shortNames.append({'ShortName' : j['ShortName'],
                                     'native-id' : r['meta']['native-id'],
                              'Version' : j['Version'],
                                'EntryTitle' : j['EntryTitle'],
                              'Count':r['meta']['granule-count'] if 'granule-count' in r['meta'] else 0,
                             'Format' : j ['ArchiveAndDistributionInformation'] if 'ArchiveAndDistributionInformation' in j else 'UNKNOWN'
                             })

i=1
for s in my_shortNames:
    print (i, s)
    i+=1
#print (my_shortNames)

before request.get
after request.get


In [15]:
r_base = 'https://cmr.earthdata.nasa.gov/search/granules.json?short_name='
i=1

cmr_granule_table = []

for s in my_shortNames:
    if s['ShortName']=='VNP14IMG_NRT':
        pass
    else:
        continue
    print (s)

    
# {'ShortName': 'AA_L2A', 
#  'native-id': 'AMSR/ADEOS-II L2A Global Swath Spatially-Resampled Brightness Temperatures V001', 
#  'EntryTitle': 'AMSR/ADEOS-II L2A Global Swath Spatially-Resampled Brightness Temperatures V001',
#  'Count': 0, 
#  'Format': {'FileDistributionInformation': [{'FormatType': 'Native', 'Format': 'HDF-EOS', 'FormatDescription': 'FTP 68 MB'}]}}
    
    r = r_base + s['ShortName']
    r += '&temporal=2021-07-01T00:00:00Z,2021-07-30T23:59:59Z'
    
    print (r)

    gJ = do_r(r, attrs=['granule_size','producer_granule_id', 'time_start', 'time_end', 'boxes', 'polygons', 'SpatialExtent'],debug=False)
    
    if gJ['count'] == 0:
        continue
        
    print ('# Granules', gJ['count'])
    for g in gJ['g']:
        print ('Example G', gJ['g'][g])
        break
    
    gn=0
    granuleSizeTotal=0.0
    
    print (i, 'Collection:', s['ShortName'], s['Count'], s)
    i+=1

    for g in gJ['g']:
        gn+=1
        gm=gJ['g'][g]
        #print(g, gJ['g'][g])
        #print(json.dumps(gm['granule_size']))
        #print ('x', json.dumps(gm))
        #print (gJ['g'][g]['title'])
        #if 'producer_granule_id' in gJ['g'][g]:  # Sometime its there, sometimes it isn't and I don't know why.
        #    fx = gJ['g'][g]['producer_granule_id'].split('.')[-1]
        #else:  # for asf don't use producer granule id
        
        if 'granule_size' in gm:
            pass
        else:
            gm['granule_size']=0.0
            
        granuleSizeTotal += float(gm['granule_size'])
        
        if gm['granule_size'] == 0.0:
          print (gm['granule_size'])
          gm['granule_size'] = 'NaN'
        
        granuleSpatial = ''
        if 'boxes' in gm and len(gm['boxes']) > 1:
            b=(str(gm['boxes']).replace("['", '').replace("']",'').replace('\n','')).split(' ')
            #print (b) # [0]lat, [1]lon, [2]lat, [3]lon
            poly_wkt='POLYGON (('
            poly_wkt+= b[1] + ' ' + b[2] + ', ' # Upper Left
            poly_wkt+= b[3] + ' ' + b[2] + ', ' # Upper Right
            poly_wkt+= b[3] + ' ' + b[0] + ', ' # Lower Right
            poly_wkt+= b[1] + ' ' + b[0] + ', ' # Lower Left
            poly_wkt+= b[1] + ' ' + b[2]        # Upper Left
            poly_wkt+= '))'
            granuleSpatial = poly_wkt
        else:
            gm['boxes']=''
        if 'polygons' in gm and len(gm['polygons'])==1:
            #print (gm['polygons'])
            p=str(gm['polygons']).replace('[', '').replace(']', '').replace("'", "").replace('\n','')
            points = p.split(' ')
            #print (points)
            ctr=0
            pair=['y','x']
            poly_wkt='POLYGON (('
            for p in points:
                ctr+=1
                #print (p)
                if ctr % 2: # latitude
                    pair[1]=p
                else:
                    pair[0]=p
                    if ctr==2:
                        poly_wkt += ' '.join(pair)
                    else:
                        poly_wkt += ', ' + ' '.join(pair) 
            poly_wkt+='))'
            gm['polygons']=poly_wkt
            granuleSpatial = poly_wkt
#            break
        else:
            gm['polygons']=''

            
        granuleSpatial=granuleSpatial.replace("'", '')
        if 'SpatialExtent' in gm: # Not present in NSIDC data
            granuleSpatial = gm['SpatialExtent']
        else:
            gm['SpatialExtent']=''
            
            
        cmr_granule_table.append ( [
            s['ShortName'],
            s['Version'],
            s['native-id'],
            s['EntryTitle'],
            gJ['g'][g]['producer_granule_id'],
            str(gJ['g'][g]['time_start']),
            str(gJ['g'][g]['time_end']),
            str(gJ['g'][g]['granule_size']),
            str(granuleSpatial)
        ])
        
        

    if gn > 0:
        print (granuleSizeTotal)


for i in cmr_granule_table:
    print (i)
    break

print ('Done')

{'ShortName': 'VNP14IMG_NRT', 'native-id': 'mmt_collection_11821', 'Version': '2', 'EntryTitle': 'VIIRS/NPP Active Fires 6-Min L2 Swath 375m - NRT', 'Count': 0, 'Format': {'FileArchiveInformation': [{'Format': 'NetCDF4', 'FormatType': 'Native', 'AverageFileSize': 2.0, 'AverageFileSizeUnit': 'MB'}], 'FileDistributionInformation': [{'Format': 'NetCDF4', 'FormatType': 'Native', 'Media': ['Online (http)'], 'AverageFileSize': 2.0, 'AverageFileSizeUnit': 'MB', 'Fees': 'No Fee'}]}}
https://cmr.earthdata.nasa.gov/search/granules.json?short_name=VNP14IMG_NRT&temporal=2021-07-01T00:00:00Z,2021-07-30T23:59:59Z
Done


In [12]:
df_cmr=pd.DataFrame(cmr_granule_table, columns=['shortname', 'version', 'native-id', 'entrytitle', 'producer_id', 'time_start', 'time_end', 'granule_size', 'granulespatial'])
df_cmr
#df_cmr['start_ts']=pd.to_datetime(df_cmr['start'],format='%Y%m%dT%H:%M:%S.%f').dt.time
#df_cmr['end_ts']  =pd.to_datetime(df_cmr['end'],format='%Y%m%dT%H:%M:%S.%f').dt.time

#df['Open'] =       pd.to_datetime(df    ['OPEN '],format= '%H:%M:%S' ).dt.time

,shortname,version,native-id,entrytitle,producer_id,time_start,time_end,granule_size,granulespatial
0,ATL02,004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATL02_20210630235813_01121209_004_02.h5,2021-06-30T23:58:12.879Z,2021-07-01T00:04:14.873Z,152.8105192184,
1,ATL02,004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATL02_20210701000415_01121210_004_02.h5,2021-07-01T00:04:14.879Z,2021-07-01T00:11:54.873Z,515.3284435272,
2,ATL02,004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATL02_20210701001155_01121211_004_02.h5,2021-07-01T00:11:54.879Z,2021-07-01T00:17:38.873Z,536.3785476685,
3,ATL02,004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATL02_20210701001739_01121212_004_02.h5,2021-07-01T00:17:38.879Z,2021-07-01T00:25:18.873Z,288.451174736,
4,ATL02,004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATLAS/ICESat-2 L1B Converted Telemetry Data V004,ATL02_20210701002519_01121213_004_02.h5,2021-07-01T00:25:18.879Z,2021-07-01T00:31:20.873Z,164.0123033524,
...,...,...,...,...,...,...,...,...,...
491935,VNP30,1,VIIRS/NPP Ice Surface Temperature 6-Min L2 Swa...,VIIRS/NPP Ice Surface Temperature 6-Min L2 Swa...,VNP30.A2021211.2330.001.2021212043626.nc,2021-07-30T23:30:00.000Z,2021-07-30T23:36:00.000Z,79.3944,"POLYGON ((-130.9723358 -30.0016136, -163.75251..."
491936,VNP30,1,VIIRS/NPP Ice Surface Temperature 6-Min L2 Swa...,VIIRS/NPP Ice Surface Temperature 6-Min L2 Swa...,VNP30.A2021211.2336.001.2021212043615.nc,2021-07-30T23:36:00.000Z,2021-07-30T23:42:00.000Z,84.7053,"POLYGON ((-138.4241943 -9.973237, -166.3986969..."
491937,VNP30,1,VIIRS/NPP Ice Surface Temperature 6-Min L2 Swa...,VIIRS/NPP Ice Surface Temperature 6-Min L2 Swa...,VNP30.A2021211.2342.001.2021212075521.nc,2021-07-30T23:42:00.000Z,2021-07-30T23:48:00.000Z,87.1009,"POLYGON ((-143.245697 10.5036945, -170.7468109..."
491938,VNP30,1,VIIRS/NPP Ice Surface Temperature 6-Min L2 Swa...,VIIRS/NPP Ice Surface Temperature 6-Min L2 Swa...,VNP30.A2021211.2348.001.2021212081221.nc,2021-07-30T23:48:00.000Z,2021-07-30T23:54:00.000Z,84.455,"POLYGON ((-146.345459 31.1860867, -177.4236145..."


In [13]:
df_cmr.to_csv('cmr_g_nsidc.tsv', sep='~', index=False)
import boto3
s3_client = boto3.client('s3')
s3_client.upload_file('cmr_g_nsidc.tsv', 'nasa-ems-sandbox', 'staging/cmr/nsidc/cmr_g_nsidc.tsv')


CMR w/ spatial
CREATE EXTERNAL TABLE `analyst_1`.`cmr_2`(
  `shortname` string COMMENT 'from deserializer', 
  `native-id` string COMMENT 'from deserializer', 
  `entrytitle` string COMMENT 'from deserializer', 
  `producer_id` string COMMENT 'from deserializer', 
  `start` string COMMENT 'from deserializer', 
  `end` string COMMENT 'from deserializer', 
  `size` string COMMENT 'from deserializer',
  `polygons` string)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ( 
  'quoteChar'='\u0000', 
  'separatorChar'='~') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://nasa-ems-sandbox/ems_analysis/ad-hoc/analyst1/cmr_2'
TBLPROPERTIES (
  'skip.header.line.count'='1', 
  'transient_lastDdlTime'='1620840672')

In [110]:
import sys
!{sys.executable} -m pip install PyAthena
from pyathena import connect
from pyathena.pandas.util import to_sql

q="""
CREATE EXTERNAL TABLE `cmr_staging.granule`(
  `shortname` string COMMENT 'from deserializer', 
  version string,
  `native-id` string COMMENT 'from deserializer', 
  `entrytitle` string COMMENT 'from deserializer', 
  `producer_id` string COMMENT 'from deserializer', 
  `begintime` string COMMENT 'from deserializer', 
  `endtime` string COMMENT 'from deserializer', 
  `size` string COMMENT 'from deserializer',
  `granuleSpatial` string
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ( 
  'quoteChar'='\u0000', 
  'separatorChar'='~') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://nasa-ems-sandbox/staging/cmr/nsidc/'
TBLPROPERTIES (
  'skip.header.line.count'='1'
)

"""
conn = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2')
pd.options.display.float_format = '{:,.4f}'.format
df = pd.read_sql(q, conn)
df


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


""


In [77]:
print(df)

Empty DataFrame
Columns: []
Index: []


In [84]:
polys=[['-0.00565261235455362 -102.03336715694878 19.071902141078052 -103.9847544919593 38.145273816730395 -106.14924959471696 57.205066144013514 -109.02797917158838 76.1924296491052 -115.72698403968063 84.13760392372362 93.45836270324006 65.3693516469824 75.38682270769895 46.32468382826406 71.70936874216318 27.255194724795338 69.34161674880907 8.17886095546597 67.32980778963457 -10.899404505724597 65.40827224419456 -29.97670680684987 63.411466320843395 -49.04860713219981 61.06971304258068 -68.09814833379474 57.33149348302798 -86.70159564447627 26.795939345351627 -73.55000389447426 -114.40528184588736 -54.519744606720806 -119.34281744470243 -35.45120858798975 -121.89575765653976 -16.37482121901807 -123.9524963864005 -0.031090394359856003 -125.60331220280027 -0.01978516853655056 -125.27957246103524 -16.363038400632124 -123.61509787601017 -35.43733169743005 -121.49849749409394 -54.50027166626624 -118.78585233705262 -73.51012826867472 -113.27136772088566 -86.5106285511061 22.373129396196088 -68.06786077457512 56.467435882825576 -49.031361259468405 60.57625409932631 -29.963656577207285 63.037831754219994 -10.88789181859205 65.07859900903064 8.190282513295823 67.00274084693316 27.267912588890816 68.97748312289289 46.341057117658906 71.2408116444422 65.39649166892605 74.61188857957319 84.24934925017939 90.45250345609466 76.23987883068152 -114.3820205940033 57.22594449552825 -108.43097810422887 38.15965029200101 -105.73771526055856 19.08386439053343 -103.6422249587782 0.00565261235455362 -101.70962744705146 -0.00565261235455362 -102.03336715694878']]

#MULTIPOLYGON (
# ((30 20, 45 40, 10 40, 30 20)),
# ((15 5, 40 10, 10 20, 5 10, 15 5))
#)
if len(polys) > 1:
    poly_wkt = 'MULTIPOLYGON('
else:
    poly_wkt='POLYGON (('

mypoly='(('

for p in polys:
    myPoly='(('
    print (p)
    p=p.replace('[', '').replace(']', '').replace("'", "").replace('\n','')
    points = p.split(' ')
    #print (points)
    ctr=0
    pair=['y','x']
    for p in points:
        ctr+=1
        print (p)
        if ctr % 2: # latitude
            pair[1]=p
        else:
            pair[0]=p
            if ctr==2:
                poly_wkt += ' '.join(pair)
            else:
                poly_wkt += ', ' + ' '.join(pair) 
poly_wkt+='))'
print (poly_wkt)


['-0.00565261235455362 -102.03336715694878 19.071902141078052 -103.9847544919593 38.145273816730395 -106.14924959471696 57.205066144013514 -109.02797917158838 76.1924296491052 -115.72698403968063 84.13760392372362 93.45836270324006 65.3693516469824 75.38682270769895 46.32468382826406 71.70936874216318 27.255194724795338 69.34161674880907 8.17886095546597 67.32980778963457 -10.899404505724597 65.40827224419456 -29.97670680684987 63.411466320843395 -49.04860713219981 61.06971304258068 -68.09814833379474 57.33149348302798 -86.70159564447627 26.795939345351627 -73.55000389447426 -114.40528184588736 -54.519744606720806 -119.34281744470243 -35.45120858798975 -121.89575765653976 -16.37482121901807 -123.9524963864005 -0.031090394359856003 -125.60331220280027 -0.01978516853655056 -125.27957246103524 -16.363038400632124 -123.61509787601017 -35.43733169743005 -121.49849749409394 -54.50027166626624 -118.78585233705262 -73.51012826867472 -113.27136772088566 -86.5106285511061 22.373129396196088 -68.

AttributeError: 'list' object has no attribute 'replace'

In [67]:
boxes="""
['-49.97828 60.697731 -27.037332 63.559301']
"""
b=(boxes.replace("['", '').replace("']",'').replace('\n','')).split(' ')
print (b) # [0]lat, [1]lon, [2]lat, [3]lon
poly_wkt='POLYGON (('
poly_wkt+= b[1] + ' ' + b[0] + ', ' # Lower Left
poly_wkt+= b[1] + ' ' + b[2] + ', '  # Upper Left
poly_wkt+= b[3] + ' ' + b[2] + ', '  # Upper Right
poly_wkt+= b[1] + ' ' + b[2] + ', '  # Lower Right
poly_wkt+= b[1] + ' ' + b[0]  # Lower Left
poly_wkt+= '))'
print (poly_wkt)


['-49.97828', '60.697731', '-27.037332', '63.559301']
POLYGON ((60.697731 -49.97828, 60.697731 -27.037332, 63.559301 -27.037332, 60.697731 -27.037332, 60.697731 -49.97828))
